# 06 — Inference (Batch Scoring)

Loads a saved pipeline from `model_registry/` and scores a CSV of new tweets.

In [ ]:

%run ./00_shared_utils.ipynb
import joblib, pandas as pd
from pathlib import Path

PIPE_PATH = Path('model_registry/logreg_calibrated_pipeline.joblib')
DATA_PATH = Path('/mnt/data/Tweets.csv')

bundle = joblib.load(PIPE_PATH)
feat_pipe = bundle['features']
clf = bundle.get('calib', bundle.get('clf'))

df_new = pd.read_csv(DATA_PATH)
X_new = df_new[['text','selected_text']]
Xfe = feat_pipe.transform(X_new)
pred = clf.predict(Xfe)
proba = clf.predict_proba(Xfe)

out = df_new.copy()
out['pred'] = pred
classes = list(clf.classes_)
for j, c in enumerate(classes):
    out[f'prob_{c}'] = proba[:, j]

out.to_csv('artifacts/scored_output.csv', index=False)
out.head()